In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [3]:
spark=SparkSession.builder\
        .appName('MySparkApp')\
        .master('local[*]')\
        .getOrCreate()
spark.version

'4.0.1'

In [13]:
users= [
    (5, "Smart Crock Pot", "2021-09-18", 698882),
    (6, "Smart Lock", "2021-09-14", 11487),
    (6, "Smart Thermostat", "2021-09-10", 674762),
    (8, "Smart Light Strip", "2021-09-29", 630773),
    (4, "Smart Cat Feeder", "2021-09-02", 693545),
    (4, "Smart Bed", "2021-09-13", 170249)
]

In [14]:
users_df=spark.createDataFrame(users,['user_id','item','created_at','amount'])
users_df.show()

+-------+-----------------+----------+------+
|user_id|             item|created_at|amount|
+-------+-----------------+----------+------+
|      5|  Smart Crock Pot|2021-09-18|698882|
|      6|       Smart Lock|2021-09-14| 11487|
|      6| Smart Thermostat|2021-09-10|674762|
|      8|Smart Light Strip|2021-09-29|630773|
|      4| Smart Cat Feeder|2021-09-02|693545|
|      4|        Smart Bed|2021-09-13|170249|
+-------+-----------------+----------+------+



In [15]:
w=Window.partitionBy('user_id').orderBy(col('created_at'))
users_df=users_df.withColumn('next_purchase_date',lead('created_at').over(w))
users_df.show()

+-------+-----------------+----------+------+------------------+
|user_id|             item|created_at|amount|next_purchase_date|
+-------+-----------------+----------+------+------------------+
|      4| Smart Cat Feeder|2021-09-02|693545|        2021-09-13|
|      4|        Smart Bed|2021-09-13|170249|              NULL|
|      5|  Smart Crock Pot|2021-09-18|698882|              NULL|
|      6| Smart Thermostat|2021-09-10|674762|        2021-09-14|
|      6|       Smart Lock|2021-09-14| 11487|              NULL|
|      8|Smart Light Strip|2021-09-29|630773|              NULL|
+-------+-----------------+----------+------+------------------+



In [16]:
users_df=users_df[date_diff(users_df['next_purchase_date'],users_df['created_at'])<=7][['user_id']]
users_df.show()

+-------+
|user_id|
+-------+
|      6|
+-------+

